rectified_text_2023_2.txt<br>
RecursiveCharacterTextSplitter<br>
jeonseonjin/embedding_BAAI-bge-m3<br>
chromadb<br>
Retrieval 과정에서 중복제거<br>
beomi/gemma-ko-2b<br>

In [1]:
#Use 3.12.4
import os
from huggingface_hub import login
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document


key_path = '/Users/jaesolshin/key/HF_TOKEN.txt'
os.environ["HF_TOKEN"] = open(key_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jaesolshin/.cache/huggingface/token
Login successful


# Document Loaders

In [2]:
path = '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'
loader = TextLoader(path)
documents = loader.load()

In [3]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 1
첫번째 문서의 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하였다. 일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승, 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설·부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭으로 축소되었다. 한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었다. 가계대출은 주택 매매 거래 증가 및 대출 규제 완화, 주택금융공사의 특례 보금자리론 등 정책 금융상품 공급 확대에 힘입어 예금은행 주택담보대출과 기타 금융기관 대출이 늘어나면서 증가 전환하였다. 다만, 높은 금리 수준 및 총부채 원리금 상환 비율(DSR, Debt Service Ratio) 규제, 지방 부동산 경기 회복 지연 등의 영향으로 예금은행 기타 대출과 비은행 예금 취급 기관은 감소세를 지속하였다.경제 동향 금융·외환시장 외환시장 원/달러 환율 상승, 변동성 소폭 축소 2023년 말 원/달러 환율(1,288.

# Text Splitting

In [4]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,   # 원하는 청크 크기 설정
    chunk_overlap=50,  # 중첩되는 토큰 수
    separators=["\n\n", "\n", ". ", "? ", "! "]  # 사용자 정의 분할 함수 추가
)

#text_splitter = CharacterTextSplitter(separator = "\n", chunk_size = 300, chunk_overlap  = 100, length_function = len,)

splitted_documents = text_splitter.split_documents(documents)

# 분할 후 각 문서의 내용을 정리
# 문장이 마침표로 시작하면 마침표 제거
# 문장이 마침표로 끝나지 않으면 마침표 추가
for i in range(len(splitted_documents)):
    splitted_documents[i].page_content = splitted_documents[i].page_content.strip('. ').strip()
    if not splitted_documents[i].page_content.endswith('.'):
        splitted_documents[i].page_content += '.'

# 분할 결과 출력
print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')



분할 전 문서 개수: 1
분할 후 문서 개수: 61


# Vector DB

In [5]:
#분할된 문장을 벡터db에 저장
#!pip install tiktoken
#!pip install chromadb==0.5.3 
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain import FAISS
import faiss
import numpy as np

# SentenceTransformer 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="jeonseonjin/embedding_BAAI-bge-m3")

# SentenceTransformer 적재를 위해 Document 객체에서 텍스트 추출
texts = [doc.page_content for doc in splitted_documents]

# 문서 임베딩 생성
embeddings = embedding_model.embed_documents(texts)

# FAISS 인덱스 생성
d = len(embeddings[0])  # 임베딩 차원 (예: 384차원)
#index = faiss.IndexFlatL2(d)  # L2 거리 기반의 FAISS 인덱스 생성
index = faiss.IndexFlatIP(d)  # 내적을 사용하여 코사인 유사도 기반 인덱스 생성
index.add(np.array(embeddings))

# LangChain의 FAISS와 연결
docstore = InMemoryDocstore({i: Document(page_content=texts[i]) for i in range(len(texts))})
database = FAISS(embedding_function=embedding_model, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(texts))})

# FAISS 인덱스가 생성되었습니다.
print("Documents added to the FAISS index.")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_1218/3344869611.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and shoul

Documents added to the FAISS index.


# Retrieval

In [6]:
query = '올해 한국은행의 정책목표 중 가장 중요한 것은 무엇인가? 답변과 함께 근거를 제시하라'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 한국은행은 이러한 정책 목표를 효과적으로 달성하는 방향으로 기준금리를 운영하고 있으며, 공개시장 운영, 여수신 제도, 지급준비 제도 등의 정책 수단을 활용하고 있다.
문서 내용: 통화신용정책 한국은행은 통화신용정책의 효율적인 수립과 집행을 통해 물가 안정을 도모하고 금융 안정에도 유의하면서 국민경제의 건전한 발전에 이바지하기 위해 최선의 노력을 기울이고 있다. 통화정책 운영체계로 물가 안정 목표제를 채택하고 있으며 동 체계하에서도 정책 결정은 물가 상황 외에 경제 성장, 금융 안정 등을 종합적으로 고려하여 이루어진다.
문서 내용: 한국은행이 수행한 통화신용정책의 상세한 내용은 한국은행이 발간하는 ｢통화신용정책보고서(분기)｣의 ｢Ⅱ. 통화신용정책 운영｣에서 확인할 수 있다. 한국은행의 통화정책 체계에 대한 상세한 설명은 한국은행이 발간한 ｢한국의 통화정책(2017)｣에서 확인할 수 있다.업무 현황 통화신용정책이라고 판단하였기 때문이다.
문서 내용: 가. 물가 안정 목표 한국은행은 2016년 이후 물가 안정 목표를 소비자 물가 상승률(전년 동기 대비) 기준 2%로 설정하고 있다. 2019년부터 적용되는 ｢2019년 이후 물가 안정 목표｣ 설정 시에는 목표 수준을 종전(2016∼18년)과 동일하게 2%로 유지하는 가운데, 물가 안정 목표의 적용 기간을 특정하지 않는 방식으로 변경하였다.
문서 내용: 한국은행은 이러한 대내외 정책 여건, 즉 물가 상승률이 기조적인 둔화 흐름을 이어갔지만 연중 물가 목표를 상회하는 오름세가 이어지고 경기와 금융안정 측면의 리스크가 상존하고 있었던 데다가 가계부채 추이, 미연준 통화정책, 지정학적 리스크 등과 관련한 높은 불확실성도 지속된 점을 고려할 때 기준금리를 긴축적인 수준에서 유지하고 그간의 3.0% 포인트 인상의 파급 효과를 점검하면서 추가 인상 필요성을 판단해 나가는 것이 필요하다.주요 이슈이다.


In [7]:
query = '최근 한국은행이 기준금리를 조정한 이유는 무엇인가요?'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 한국은행은 이러한 정책 목표를 효과적으로 달성하는 방향으로 기준금리를 운영하고 있으며, 공개시장 운영, 여수신 제도, 지급준비 제도 등의 정책 수단을 활용하고 있다.
문서 내용: 한국은행은 이러한 대내외 정책 여건, 즉 물가 상승률이 기조적인 둔화 흐름을 이어갔지만 연중 물가 목표를 상회하는 오름세가 이어지고 경기와 금융안정 측면의 리스크가 상존하고 있었던 데다가 가계부채 추이, 미연준 통화정책, 지정학적 리스크 등과 관련한 높은 불확실성도 지속된 점을 고려할 때 기준금리를 긴축적인 수준에서 유지하고 그간의 3.0% 포인트 인상의 파급 효과를 점검하면서 추가 인상 필요성을 판단해 나가는 것이 필요하다.주요 이슈이다.
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다양한 공개시장 운영 수단을 활용하여 유동성을 신축적으로 조절하였다. 아울러 공개시장 운영 여건 변화에 맞춰 통화안정증권 발행 제도를 개선함으로써 유동성 조절의 효율성을 제고하였다.
문서 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다.
문서 내용: 나. 기준금리 한국은행은 2023년 중 성장세를 점검하면서 중기적 시계에서 물가 상승률이 목표 수준(2%)에서 안정될 수 있도록 기준금리를 긴축적인 수준에서 운용하였다.


In [8]:
query = '한국은행의 통화정책과 금융안정 정책의 차이점은 무엇인가요?'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 통화신용정책 한국은행은 통화신용정책의 효율적인 수립과 집행을 통해 물가 안정을 도모하고 금융 안정에도 유의하면서 국민경제의 건전한 발전에 이바지하기 위해 최선의 노력을 기울이고 있다. 통화정책 운영체계로 물가 안정 목표제를 채택하고 있으며 동 체계하에서도 정책 결정은 물가 상황 외에 경제 성장, 금융 안정 등을 종합적으로 고려하여 이루어진다.
문서 내용: 한국은행이 수행한 통화신용정책의 상세한 내용은 한국은행이 발간하는 ｢통화신용정책보고서(분기)｣의 ｢Ⅱ. 통화신용정책 운영｣에서 확인할 수 있다. 한국은행의 통화정책 체계에 대한 상세한 설명은 한국은행이 발간한 ｢한국의 통화정책(2017)｣에서 확인할 수 있다.업무 현황 통화신용정책이라고 판단하였기 때문이다.
문서 내용: 한국은행은 통화신용정책 등을 수행함에 있어 금융기관 등을 대상으로 여수신 업무를 수행한다. 2023년 중 운용한 여신 업무로는 금융중개지원대출, 자금조정대출, 일중당좌대출 등이 있으며, 수신 업무로는 당좌예금, 자금조정예금 및 통화 안정계정 예치금 등이 있다.
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다양한 공개시장 운영 수단을 활용하여 유동성을 신축적으로 조절하였다. 아울러 공개시장 운영 여건 변화에 맞춰 통화안정증권 발행 제도를 개선함으로써 유동성 조절의 효율성을 제고하였다.
문서 내용: 입력: 23 Ⅰ 경제 동향 2 금융･외환시장 QR BOX QR1 통화신용정책보고서 웹페이지 QR2 경제전망보고서 웹페이지 QR3 금융안정보고서 웹페이지 QR4 지역경제보고서 웹페이지 QR5 한국은행 블로그  
출력:주요 이슈.


In [9]:
query = '한국은행의 최근 금융시장 안정화 조치는 무엇이었나요?'

results = database.similarity_search(query, k=5) 

# 답변에서 중복 제거
unique_results = {result.page_content: result for result in results}.values()

print(f'문서 개수: {len(unique_results)}')

for d in unique_results:
    print(f'문서 내용: {d.page_content}')

문서 개수: 5
문서 내용: 통화신용정책 한국은행은 통화신용정책의 효율적인 수립과 집행을 통해 물가 안정을 도모하고 금융 안정에도 유의하면서 국민경제의 건전한 발전에 이바지하기 위해 최선의 노력을 기울이고 있다. 통화정책 운영체계로 물가 안정 목표제를 채택하고 있으며 동 체계하에서도 정책 결정은 물가 상황 외에 경제 성장, 금융 안정 등을 종합적으로 고려하여 이루어진다.
문서 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다.
문서 내용: 한국은행은 이러한 정책 목표를 효과적으로 달성하는 방향으로 기준금리를 운영하고 있으며, 공개시장 운영, 여수신 제도, 지급준비 제도 등의 정책 수단을 활용하고 있다.
문서 내용: 한국은행은 통화신용정책 등을 수행함에 있어 금융기관 등을 대상으로 여수신 업무를 수행한다. 2023년 중 운용한 여신 업무로는 금융중개지원대출, 자금조정대출, 일중당좌대출 등이 있으며, 수신 업무로는 당좌예금, 자금조정예금 및 통화 안정계정 예치금 등이 있다.
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다양한 공개시장 운영 수단을 활용하여 유동성을 신축적으로 조절하였다. 아울러 공개시장 운영 여건 변화에 맞춰 통화안정증권 발행 제도를 개선함으로써 유동성 조절의 효율성을 제고하였다.


# Query Augmentation

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline


model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="mps",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Hugging Face pipeline을 사용하여 LLM 구성
hf_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_length=1024,  
    temperature=0.7, 
    top_k=50,  
    top_p=0.9,
    do_sample=True, 
    truncation=True,
)

# LangChain에서 Hugging Face 모델을 LLM으로 사용
huggingface_llm = HuggingFacePipeline(pipeline=hf_pipeline)

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_1218/703145491.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  huggingface_llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [12]:
from langchain.chains import RetrievalQA

# Retriever 설정
retriever = database.as_retriever()  # 데이터베이스를 Retriever로 변환

get_answer = RetrievalQA.from_llm( #RetrievalQA를 초기화
    llm=huggingface_llm, #Chat models를 지정
    retriever = retriever, #Retriever를 지정
    return_source_documents=False #응답에 원본 문서를 포함할지 결정
)

query = "올해 한국은행의 정책목표 중 가장 중요한 것은 무엇인가?"

def answer_to_query(query):
    answer = get_answer(query)
    if 'Helpful Answer:\n' in answer['result']:
        parts = answer['result'].split('Helpful Answer:\n')
        if len(parts) > 2:
            answer_parsed = parts[1].strip()  # 첫 번째와 두 번째 사이의 내용
        else:
            answer_parsed = parts[-1].strip()  # 한 번만 등장하면 그 이후의 내용
    elif 'Answer:' in answer['result']:
        answer_parsed = answer['result'].split('Answer:')[-1].strip()
    else:
        answer_parsed = answer['result'].strip()
    return answer_parsed

answer_to_query(query)

RuntimeError: no validator found for <class 'langchain_core.prompts.base.BasePromptTemplate'>, see `arbitrary_types_allowed` in Config

In [ ]:
answer_to_query('최근 한국은행이 기준금리를 조정한 이유는 무엇인가요?')

'기준금리 운용.'

In [ ]:
get_answer('한국은행과 중앙정부 사이 갈등요인은 무엇인가?')['result']

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:\n한국은행은 개발도상국 중앙은행을 대상으로 통화정책, 지급결제, 금융안정 등 중앙은행 고유 업무와 관련된 다양한 정책 경험을 전수함으로써 이들 국가의 금융 발전을 지원하고 중앙은행 간의 우호적인 협력관계를 강화하였다.\n\nContext:\n한국은행은 개발도상국 중앙은행을 대상으로 통화정책, 지급결제, 금융안정 등 중앙은행 고유 업무와 관련된 다양한 정책 경험을 전수함으로써 이들 국가의 금융 발전을 지원하고 중앙은행 간의 우호적인 협력관계를 강화하였다.\n\nContext:\n한국은행은 개발도상국 중앙은행을 대상으로 통화정책, 지급결제, 금융안정 등 중앙은행 고유 업무와 관련된 다양한 정책 경험을 전수함으로써 이들 국가의 금융 발전을 지원하고 중앙은행 간의 우호적인 협력관계를 강화하였다.\n\nContext:\n한국은행은 공정한 채용절차를 통해 중앙은행 직원으로서의 사명감, 조직융화력 등 기본 소양은 물론 전문지식 및 직무역량을 겸비한 인재를 선발하기 위한 노력을 지속하였으며 사회적 기대에 부응하기 위한 인력운영 노력을 병행하였다.\n\nQuestion: 한국은행과 중앙정부 사이 갈등요인은 무엇인가?\nHelpful Answer:\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇인가?\n한국은행과 중앙정부 사이 갈등요인은 무엇

In [2]:
import pkg_resources

installed_packages = pkg_resources.working_set
packages = sorted((package.project_name, package.version) for package in installed_packages)

# 여러 단으로 출력
for i, (name, version) in enumerate(packages):
    print(f"{name}=={version}")
    if (i + 1) % 5 == 0:  # 5개마다 줄 바꿈
        print()  # 줄 바꿈

Babel==2.15.0
Brotli==1.1.0
CurrencyConverter==0.17.30
Cython==3.0.11
Deprecated==1.2.14

Faker==28.4.1
GitPython==3.1.43
GridDataFormats==1.0.2
Lazify==0.4.0
MDAnalysis==2.7.0

Markdown==3.6
MarkupSafe==2.1.5
MouseInfo==0.1.3
PyAutoGUI==0.9.54
PyGetWindow==0.0.9

PyJWT==2.9.0
PyMsgBox==1.0.9
PyMuPDF==1.24.10
PyMuPDFb==1.24.10
PyPika==0.48.9

PyRect==0.2.0
PyScreeze==1.0.1
PySocks==1.7.1
PyYAML==6.0.2
QtPy==2.4.1

SQLAlchemy==2.0.32
Send2Trash==1.8.3
TPOT==0.12.2
absl-py==2.1.0
accelerate==0.34.2

aiofiles==23.2.1
aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
altair==5.4.1

annotated-types==0.7.0
anyio==3.7.1
appnope==0.1.4
appscript==1.2.5
argon2-cffi==23.1.0

argon2-cffi-bindings==21.2.0
arrow==1.3.0
ase==3.23.0
asgiref==3.8.1
asttokens==2.4.1

astunparse==1.6.3
async-lru==2.0.4
asyncer==0.0.7
attrs==24.2.0
auth0-python==4.7.1

autocommand==2.2.2
backoff==2.2.1
backports.tarfile==1.2.0
bcrypt==4.2.0
beautifulsoup4==4.12.3

bidict==0.23.1
bitsandbytes==0.42.0
bleach==6.1.0


/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_1622/2112360723.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
